Преобразование таблицы с исправлением ошибок.

In [7]:
import pandas as pd
from datetime import datetime

file_path = 'PathToFolder/data.xlsx'
data = pd.read_excel(file_path)

current_month_year = None
month_year_list = []

months_mapping = {
    "Январь": "January",
    "Февраль": "February",
    "Март": "March",
    "Апрель": "April",
    "Май": "May",
    "Июнь": "June",
    "Июль": "July",
    "Август": "August",
    "Сентябрь": "September",
    "Октябрь": "October",
    "Ноябрь": "November",
    "Декабрь": "December"
}

def translate_month_year(month_year):
    month, year = month_year.split()
    return datetime.strptime(f"{months_mapping[month]} {year}", "%B %Y")

for index, row in data.iterrows():
    if pd.isna(row['client_id']) and pd.isna(row['sum']):
        current_month_year = row['status']
    month_year_list.append(current_month_year)

data['month_and_year'] = month_year_list

data_cleaned = data.dropna(subset=['client_id', 'sum'])

data_cleaned = data_cleaned.drop(columns=['Unnamed: 5'])

data_cleaned['month_and_year'] = data_cleaned['month_and_year'].apply(translate_month_year)

data_cleaned

     client_id      sum      status        sale new/current  document  \
1          6.0  11693.7  ВНУТРЕННИЙ           -           -         -   
2         14.0   5452.7    ОПЛАЧЕНО     Петрова       новая  оригинал   
3         15.0  13991.7    ОПЛАЧЕНО      Иванов     текущая  оригинал   
4         16.0    719.7    ОПЛАЧЕНО      Иванов     текущая  оригинал   
5         18.0   4253.7    ОПЛАЧЕНО      Иванов     текущая  оригинал   
..         ...      ...         ...         ...         ...       ...   
725      285.0   4328.7    ОПЛАЧЕНО    Васильев     текущая  оригинал   
726      360.0   3096.2    ОПЛАЧЕНО    Васильев       новая       НЕТ   
727      360.0   3096.2  ПРОСРОЧЕНО    Васильев       новая       НЕТ   
728      373.0   4403.7    ОПЛАЧЕНО    Михайлов       новая  оригинал   
729      286.0   3978.7    ОПЛАЧЕНО  Филимонова     текущая  оригинал   

          receiving_date month_and_year  
1                      -     2021-05-01  
2    2021-07-26 00:00:00     2021-05-01

Расчёт остатка каждого из менеджеров на 01.07.2021.

In [6]:
def calculate_bonus_and_residual(row, current_month_year):
    if row['document'] == 'оригинал' and row['status'] == 'ОПЛАЧЕНО' and row['new/current'] == 'новая':
        if row['month_and_year'] == current_month_year:
            return row['sum'] * 0.07, 0
        else:
            return 0, row['sum'] * 0.07
    elif row['document'] == 'оригинал' and row['status'] != 'ПРОСРОЧЕНО' and row['new/current'] == 'текущая':
        if row['sum'] > 10000:
            bonus = row['sum'] * 0.05
        else:
            bonus = row['sum'] * 0.03
        
        if row['month_and_year'] == current_month_year:
            return bonus, 0
        else:
            return 0, bonus
    else:
        return 0, 0

cutoff_date = datetime.strptime('2021-07-01', '%Y-%m-%d')

data_cleaned['bonus'], data_cleaned['residual'] = zip(*data_cleaned.apply(calculate_bonus_and_residual, current_month_year=cutoff_date, axis=1))

residuals = data_cleaned[data_cleaned['month_and_year'] < cutoff_date].groupby('sale')['residual'].sum().reset_index()

residuals

,sale,residual
0,-,0.0000
1,Андреев,8744.0334
2,Васильев,1376.2800
3,Иванов,8495.2910
4,Кузнецова,7101.6217
5,Петрова,17799.5494
6,Селиванов,5651.0120
7,Смирнов,10508.9890
8,Соколов,269.2110
9,Филимонова,2727.6409
